### Output Check

In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from numpy import asarray
from tensorflow.keras.layers import Input,Embedding,GRU,MaxPooling2D,GlobalMaxPooling2D,Bidirectional
from tensorflow.keras.layers import LSTM,Concatenate,MaxPool1D,Flatten,Dropout,Masking,Softmax,Dense,Embedding
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,TerminateOnNaN,ReduceLROnPlateau,LearningRateScheduler
from tensorflow.keras.models import Sequential
#BPEmb is set of pretrained vectors like glove vectors
from bpemb import BPEmb
from tqdm import tqdm

In [2]:
DfTest = pd.read_pickle('PreProcessed\WordLevelTestingData.pkl')

In [3]:
#defining Metric
def perplexity(y_true, y_pred):
    '''
    https://thegradient.pub/understanding-evaluation-metrics-for-language-models/
    return perplexity of the module
    The perplexity of a language model can be seen as the level of perplexity when predicting the following symbol    
    '''
    return tf.keras.backend.exp(tf.keras.backend.mean(tf.keras.backend.sparse_categorical_crossentropy(y_true, y_pred)))

In [4]:
#declaring bpemb_en
bpemb_en = BPEmb(lang="en",vs=50000)
bpemb_en.vectors.shape

(50000, 100)

In [5]:
#Loading Model
loadModel = tf.keras.models.load_model('model2_save/weights-24-5.3944.hdf5',custom_objects = {'perplexity':perplexity})

In [6]:
def predictOutput(input_sentence):
    '''
    This Method returns the output sentence for given input sentence
    In this method we pass input sentence through all the layers in the model
    '''
    #Tokenizing input sentence
    enCode = bpemb_en.encode_ids_with_bos_eos(input_sentence)
    deSentence = [[1]]
    encInpTrain = pad_sequences([enCode], maxlen=15, padding="post")
    decoder_input_train = np.array(deSentence)
    embEncoder = loadModel.layers[1](encInpTrain)
    encOp,stateF,stateB=loadModel.layers[3](embEncoder)
    concatenated = loadModel.layers[5]([stateF,stateB])
    outputSentenceList = []
    for i in range(0,15):
        embeddingDec = loadModel.layers[4](decoder_input_train)
        output,state = loadModel.layers[6](embeddingDec, initial_state = concatenated)
        dense = loadModel.layers[8](output)
        dense1 = loadModel.layers[10](dense)
        index = np.argmax(dense1[0][0])
        deSentence = [[int(index)]]
        decoder_input_train = np.array(deSentence)
        outputSentenceList.append(int(index))
        concatenated = state
        if(index == 0):
              break
    return outputSentenceList

In [7]:
def returnOutput(listData):
    '''
    This method passes input sentences in the list form to predictOutput function
    '''
    Predictions = []
    Lst = []
    for txt in tqdm(listData):
        Lst = bpemb_en.decode_ids(predictOutput(txt))
        Predictions.append(' '.join(Lst))
    return Predictions

In [8]:
#Loading random data
TestDataFrame = pd.DataFrame()
TestDataFrame['Input Sentence']= DfTest['X'].iloc[560:800]
TestDataFrame['Actual Output Sentence']= DfTest['Y'].iloc[560:800]
TestDataFrame['Predicted Output Sentence']= returnOutput(TestDataFrame['Input Sentence'].values)

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [00:23<00:00, 10.04it/s]


In [9]:
TestDataFrame.head(5)

,Input Sentence,Actual Output Sentence,Predicted Output Sentence
560,i will pass along any revised numbers as we,receive them,have a copy
561,also let anyone you want,to look at it as well,to get a copy of the attached
562,thank you for attending the,meeting and giving many helpful contributions,meeting
563,they have been sniffing,around the marketplace for several weeks now,the same
564,there is some,though that amos might not run again,of the same


In [10]:
TestDataFrame[50:55]

,Input Sentence,Actual Output Sentence,Predicted Output Sentence
610,i should have stated,that vick is the perfect example,the same
611,please let me know if,you have any problems or questions,you have any questions
612,i thought you might find some,aspects interesting,to be a good time
613,you will be presented with a winzip,dialog box,of the office
614,however i will,be out of the office until monday,be able to get a copy of the attached


In [18]:
def returnOutputText(txt):
    '''
    This method passes input sentences predictOutput function
    '''
    oupSent = bpemb_en.decode_ids(predictOutput(txt))
    oupSent = ' '.join(oupSent)
    return oupSent

In [20]:
returnOutputText("thank you for attending the")

'meeting  '